In [ ]:
import numpy as np
from landlab import RasterModelGrid, imshow_grid
import os
from landlab.grid.mappers import map_downwind_node_link_mean_to_node
import matplotlib.pyplot as plt
import glob
import natsort
import pandas as pd

In [ ]:
a = np.loadtxt('infinite-pit-trans_1_flux-ts=3000.txt')
rows = 299
cols = 299
grid = RasterModelGrid((rows, cols), xy_spacing = 0.025)
grid.at_link['flux'] = a
flux = map_downwind_node_link_mean_to_node(grid, 'flux')

imshow_grid(grid, flux)

In [ ]:
directory = 'fluxes'

fluxes = np.zeros((10, 62))
circle_radius = [0.25, 0.25, 0.25]
circle_center = (299*0.025/2, 299*0.025/2)  # Center of the circle, adjust as needed

count = 0
for d in natsort.natsorted(os.listdir(directory)):
    if os.path.isdir(os.path.join(directory,d)) == True and os.path.join(directory,d) != '2_infinite-pits-transects\.ipynb_checkpoints':
        i = 0
        print(os.path.join(directory,d))
        for f in natsort.natsorted(glob.glob(os.path.join(directory, d, "*.txt"))):
            a = np.loadtxt(f)

            rows = 299
            cols = 299
            grid = RasterModelGrid((rows, cols), xy_spacing = 0.025)
            grid.at_link['flux'] = a
            flux = map_downwind_node_link_mean_to_node(grid, 'flux')
            
            x_m = 299*0.025/2
            y_m = 299*0.025/2

            dists = grid.calc_distances_of_nodes_to_point([x_m, y_m])

            nodes_on_circumference = []
            j = 0
            for distance in dists:
                if (distance <= circle_radius[count]+0.025) and (distance > circle_radius[count]-0.025):
                    nodes_on_circumference.append(j)
                j += 1
                
            circ_flux = np.array([flux[k] for k in nodes_on_circumference])

            fluxes[count][i] = np.mean(circ_flux)
            i += 1

        count += 1

In [ ]:
time = np.arange(0, 61) * 0.5
colors = ["#D55E00", "#0072B2", "#CC79A7", "#F0E442", "#D55E00"]

plt.plot(time, fluxes[0][:-1], label = "$\\ell = 0.029$ m", color = colors[0], linewidth = 3, linestyle = 'solid')
plt.plot(time, fluxes[1][:-1], label = "$\\ell = 0.144$ m", color = colors[1], linewidth = 3, linestyle = 'dashed')
plt.plot(time, fluxes[2][:-1], label = "$\\ell = 0.289$ m", color = colors[2], linewidth = 3, linestyle = 'dashdot')

plt.xlabel('Time (days)')
plt.ylabel('$\\overline{Q}$ (m$^2$/day)')
plt.legend(title = "Length Scale")

In [ ]:
ice1 = 'infinite-pit-trans_1_s_trans.csv'
topo1 = 'infinite-pit-trans_1_z_trans.csv'
ice2 = 'infinite-pit-trans_2_s_trans.csv'
topo2 = 'infinite-pit-trans_2_z_trans.csv'
ice3 = 'infinite-pit-trans_3_s_trans.csv'
topo3 = 'infinite-pit-trans_3_z_trans.csv'

ice1df = pd.read_csv(ice1)
topo1df = pd.read_csv(topo1)
ice2df = pd.read_csv(ice2)
topo2df = pd.read_csv(topo2)
ice3df = pd.read_csv(ice3)
topo3df = pd.read_csv(topo3)

In [ ]:
plt.plot(topo1df['x']-3.75, topo1df['30.0']-98.8, color = 'k', label = "$\\ell = 0.029$ m", linestyle = 'dashed')
plt.plot(ice1df['x']-3.75, ice1df['30.0']-98.8, color = 'steelblue', linestyle = 'dashed')

plt.plot(topo3df['x']-3.75, topo3df['30.0']-98.8, color = 'k', linestyle = 'solid', label = "$\\ell = 0.289$ m")
plt.plot(ice3df['x']-3.75, ice3df['30.0']-98.8, color = 'steelblue', linestyle = 'solid')
plt.xlim([0, 1.5])
plt.ylim([-0.05, 0.9])
plt.xlabel('$x$ (m)', fontsize = 11)
plt.ylabel("$z'$ (m)", fontsize = 11)
plt.annotate('30 days\n$b_0$ = 0.04 m/day\nD = 0.001 m$^2$/day\n$\ell = 0.029$ m', xycoords = 'axes fraction', xy = (0.15, 0.78))
plt.annotate('30 days\n$b_0$ = 0.04 m/day\nD = 0.01 m$^2$/day\n$\ell = 0.289$ m', xycoords = 'axes fraction', xy = (0.6, 0.23))
plt.annotate('$\\ell = \\frac{DS_c}{b_0}$', xycoords = 'axes fraction', xy = (0.65, 0.84), fontsize = 20)
#plt.legend()
#plt.axis('equal')
#plt.savefig('infinite-pits-debris.png', dpi = 300)
plt.show()

In [ ]:
deb1 = np.array(topo1df.iloc[161] - ice1df.iloc[161])
deb2 = np.array(topo2df.iloc[161] - ice2df.iloc[161])
deb3 = np.array(topo3df.iloc[161] - ice3df.iloc[161])

melt1 = 0.04 * 0.08 / (0.08 + deb1)
melt2 = 0.04 * 0.08 / (0.08 + deb2)
melt3 = 0.04 * 0.08 / (0.08 + deb3)

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
plt.plot(topo1df['x']-3.75, savgol_filter(topo1df['30.0'], window_length = 3, polyorder = 1, deriv = 1, delta = 0.025))
plt.plot(topo2df['x']-3.75, savgol_filter(topo2df['30.0'], window_length = 3, polyorder = 1, deriv = 1, delta = 0.025))
plt.plot(topo3df['x']-3.75, savgol_filter(topo3df['30.0'], window_length = 3, polyorder = 1, deriv = 1, delta = 0.025))
plt.xlim([0, 1.5])
plt.ylim([-1.4, 0.1])
plt.axhline(-1.15, color = 'k', linestyle = 'dashed')
#plt.axvline(3.95-3.75, color = 'k', linewidth = 0.5)
plt.axvline(4.025-3.75, color = 'k', linewidth = 0.5)

In [ ]:
colors = ["#D55E00", "#0072B2", "#CC79A7", "#F0E442", "#D55E00"]
time = np.arange(0, 61) * 0.5
fig, ax = plt.subplots(3, 1, figsize = (4, 8))


ax[0].plot(time, fluxes[0][:-1], label = "$\\ell = 0.029$ m", color = colors[0], linewidth = 3, linestyle = 'solid')
ax[0].plot(time, fluxes[1][:-1], label = "$\\ell = 0.144$ m", color = colors[1], linewidth = 3, linestyle = 'dashed')
ax[0].plot(time, fluxes[2][:-1], label = "$\\ell = 0.289$ m", color = colors[2], linewidth = 3, linestyle = 'dashdot')
ax[0].set_ylabel("$\\overline{Q}$ (m$^2$/day)", fontsize = 11)
ax[0].legend()

ax[1].plot(np.arange(0, len(deb1[1:]))*0.5, deb1[1:], color = colors[0], linewidth = 3, linestyle = 'solid', label = "$\\ell = 0.029$ m")
ax[1].plot(np.arange(0, len(deb2[1:]))*0.5, deb2[1:], color = colors[1], linewidth = 3, linestyle = 'dashed', label = "$\\ell = 0.144$ m")
ax[1].plot(np.arange(0, len(deb3[1:]))*0.5, deb3[1:], color = colors[2], linewidth = 3, linestyle = 'dashdot', label = "$\\ell = 0.289$ m")
ax[1].set_ylabel("Debris Thickness (m)", fontsize = 11)
ax[1].legend()
#ax[1].annotate('$\ell = 0.029$ m', xycoords = 'axes fraction', xy = (0.28, 0.94), fontsize = 8)
#ax[1].annotate('$\ell = 0.144$ m', xycoords = 'axes fraction', xy = (0.5, 0.72), fontsize = 8)
#ax[1].annotate('$\ell = 0.289$ m', xycoords = 'axes fraction', xy = (0.7, 0.52), fontsize = 8)

ax[2].plot(np.arange(0, len(melt1[1:]))*0.5, melt1[1:], color = colors[0], linewidth = 3, linestyle = 'solid', label = "$\\ell = 0.029$ m")
ax[2].plot(np.arange(0, len(melt2[1:]))*0.5, melt2[1:], color = colors[1], linewidth = 3, linestyle = 'dashed', label = "$\\ell = 0.144$ m")
ax[2].plot(np.arange(0, len(melt3[1:]))*0.5, melt3[1:], color = colors[2], linewidth = 3, linestyle = 'dashdot', label = "$\\ell = 0.289$ m")
ax[2].set_ylabel("Melt Rate (m/day)", fontsize = 11)
ax[2].set_xlabel('Days', fontsize = 11)
ax[2].legend()
#ax[2].annotate('$\ell = 0.029$ m', xycoords = 'axes fraction', xy = (0.75, 0.09), fontsize = 8)
#ax[2].annotate('$\ell = 0.144$ m', xycoords = 'axes fraction', xy = (0.65, 0.27), fontsize = 8)
#ax[2].annotate('$\ell = 0.289$ m', xycoords = 'axes fraction', xy = (0.5, 0.35), fontsize = 8)

ax[0].annotate("(a)", xy = (-0.25, 1.05), xycoords = "axes fraction", fontsize = 12)
ax[1].annotate("(b)", xy = (-0.25, 1.05), xycoords = "axes fraction", fontsize = 12)
ax[2].annotate("(c)", xy = (-0.25, 1.05), xycoords = "axes fraction", fontsize = 12)

plt.tight_layout()
#plt.savefig('infinite-pits-flux-h.png', dpi = 300)
plt.show()
